In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain.graphs import Neo4jGraph

url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password
)

In [3]:
import requests

url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/da8882249af3e819a80debf3160ebbb3513ee962/microservices.json"
import_query = requests.get(url).json()['query']
graph.query(
    import_query
)

[]

In [3]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-l6-v2")

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    index_name='tasks',
    node_label=["Task","Team",'Microservice'],
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

/Users/himankjain/Desktop/Langchain/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
response = vector_index.similarity_search(
    "How will recommendation service be updated?"
)
print(response[0].page_content)


name: RecommendationFeature
description: Add feature to RecommendationService
status: In Progress


In [6]:
from langchain_community.chat_models import AzureChatOpenAI

key = os.getenv('OPENAI_API_KEY')
endpoint = os.getenv('OPENAI_API_ENDPOINT')

gpt4 = AzureChatOpenAI(
    deployment_name ='<name>',
    openai_api_version = "<version>",
     openai_api_key = key,
    azure_endpoint = endpoint,
    temperature = 0,
    max_tokens = 2000,
    request_timeout = 60,
    max_retries = 3
)

/Users/himankjain/Desktop/Langchain/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.chains import RetrievalQA

vector_qa = RetrievalQA.from_chain_type(
    llm=gpt4,
    chain_type="stuff",
    retriever=vector_index.as_retriever(),
    verbose=True
)

In [8]:
vector_qa.run(
    "How will recommendation service be updated?"
)

/Users/himankjain/Desktop/Langchain/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



> Finished chain.


'The RecommendationService is currently being updated with the addition of a new feature called RecommendationFeature. The status of this update is "In Progress," which means it is still being worked on and not yet completed.'

In [12]:
vector_qa.run(
    "What is BugFix task about?"
)



> Entering new RetrievalQA chain...

> Finished chain.


'The BugFix task is about fixing a bug in the CatalogService.'

In [16]:
vector_qa.run(
    "What is CatalogService?"
)



> Entering new RetrievalQA chain...

> Finished chain.


'CatalogService is not explicitly defined in the provided context. However, it appears to be a service related to a software system or application, possibly dealing with managing or organizing a catalog of items or products. The context mentions a task called "BugFix" which aims to fix a bug in CatalogService, and its status is currently "Open."'

In [17]:
vector_qa.run(
    "CatalogService depends on which microservice?"
)



> Entering new RetrievalQA chain...

> Finished chain.


"I don't have enough information to determine which microservice CatalogService depends on."

In [18]:
vector_qa.run(
    "How many 'In Progress' tickets are there?"
)



> Entering new RetrievalQA chain...

> Finished chain.


"There are 2 'In Progress' tickets."

In [21]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = gpt4,
    qa_llm = gpt4, graph=graph, verbose=True,
)

In [22]:
cypher_chain.run(
    "How many 'In Progress' tickets there are?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task {status: 'In Progress'}) RETURN COUNT(t)
Full Context:
[{'COUNT(t)': 3}]

> Finished chain.


"There are 3 'In Progress' tickets."

In [23]:
cypher_chain.run(
    "CatalogService depends on which microservice?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice {name: 'CatalogService'})-[:DEPENDS_ON]->(ms:Microservice) RETURN ms.name
Full Context:
[{'ms.name': 'ReviewService'}, {'ms.name': 'RecommendationService'}]

> Finished chain.


'CatalogService depends on ReviewService and RecommendationService microservices.'

In [24]:
cypher_chain.run(
    "Who is the leader of TeamD?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:LEAD_OF]->(t:Team {name: "TeamD"}) RETURN p.name
Full Context:
[{'p.name': 'Diana'}]

> Finished chain.


'Diana is the leader of TeamD.'

In [27]:
cypher_chain.run(
    "'Enhance' task is assigned to which Team and linked to which Microservice?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task)-[:ASSIGNED_TO]->(team:Team), (t)-[:LINKED_TO]->(microservice:Microservice) WHERE t.name = 'Enhance' RETURN team, microservice;
Full Context:
[{'team': {'name': 'TeamB'}, 'microservice': {'name': 'ShippingService', 'technology': 'Python'}}]

> Finished chain.


"The 'Enhance' task is assigned to TeamB and linked to the ShippingService microservice."

In [28]:
cypher_chain.run(
    "'Update' task is assigned to which Team and linked to which Microservice? Who owns the Microservice"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task {name: 'Update'})-[:ASSIGNED_TO]->(team:Team), (t)-[:LINKED_TO]->(ms:Microservice), (ms)-[:MAINTAINED_BY]->(owner:Team)
RETURN team, ms, owner
Full Context:
[{'team': {'name': 'TeamA'}, 'ms': {'name': 'InventoryService', 'technology': 'Java'}, 'owner': {'name': 'TeamA'}}]

> Finished chain.


"The 'Update' task is assigned to TeamA and linked to the InventoryService microservice. TeamA owns the Microservice."

In [30]:
cypher_chain.run(
    "How many Microservices are owned by TeamA? Enlist them"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Team {name: "TeamA"})-[:MAINTAINED_BY]-(m:Microservice) RETURN COUNT(m) as MicroservicesOwned, COLLECT(m.name) as MicroserviceNames
Full Context:
[{'MicroservicesOwned': 3, 'MicroserviceNames': ['CatalogService', 'InventoryService', 'AuthService']}]

> Finished chain.


'TeamA owns 3 Microservices, which are CatalogService, InventoryService, and AuthService.'

In [41]:
cypher_chain.run(
    "List all services that use Node.js or Java"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice) WHERE m.technology IN ['Node.js', 'Java'] RETURN m.name, m.technology
Full Context:
[{'m.name': 'CatalogService', 'm.technology': 'Java'}, {'m.name': 'PaymentService', 'm.technology': 'Node.js'}, {'m.name': 'InventoryService', 'm.technology': 'Java'}, {'m.name': 'RecommendationService', 'm.technology': 'Node.js'}, {'m.name': 'AuthService', 'm.technology': 'Node.js'}]

> Finished chain.


'The services that use Node.js or Java are CatalogService (Java), PaymentService (Node.js), InventoryService (Java), RecommendationService (Node.js), and AuthService (Node.js).'

In [31]:
cypher_chain.run(
    "Find name of microservices that has dependency on name 'Database' directly?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice)-[:DEPENDS_ON]->(d:Dependency {name: 'Database'}) RETURN m.name
Full Context:
[{'m.name': 'CatalogService'}, {'m.name': 'OrderService'}, {'m.name': 'UserService'}, {'m.name': 'PaymentService'}, {'m.name': 'InventoryService'}, {'m.name': 'AuthService'}]

> Finished chain.


"I don't know the answer."

In [32]:
cypher_chain.run(
    "'Database' is a dependency for which microservices?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice)-[:DEPENDS_ON]->(d:Dependency) WHERE d.name = 'Database' RETURN m.name
Full Context:
[{'m.name': 'CatalogService'}, {'m.name': 'OrderService'}, {'m.name': 'UserService'}, {'m.name': 'PaymentService'}, {'m.name': 'InventoryService'}, {'m.name': 'AuthService'}]

> Finished chain.


"I am not sure about the 'Database' dependency, but the available microservices are CatalogService, OrderService, UserService, PaymentService, InventoryService, and AuthService."

In [36]:
cypher_chain.run(
    "What is the 'type' of 'Database'?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Dependency {name: 'Database'}) RETURN d.type
Full Context:
[{'d.type': 'SQL'}]

> Finished chain.


'The type of database is SQL.'

AGENTS

In [42]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Tasks",
        func=vector_qa.run,
        description="""Useful when you need to answer questions about descriptions of tasks.
        Not useful for counting the number of tasks.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.run,
        description="""Useful when you need to answer questions about microservices,
        their dependencies or assigned people. Also useful for any sort of 
        aggregation like counting the number of tasks, etc.
        Use full question as input.
        """,
    ),
]

mrkl = initialize_agent(
    tools, 
    gpt4,
    agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

/Users/himankjain/Desktop/Langchain/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [43]:
response = mrkl.run("PaymentService is maintained by which team?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `PaymentService is maintained by which team?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice {name: "PaymentService"})-[:MAINTAINED_BY]->(t:Team) RETURN t.name
Full Context:
[{'t.name': 'TeamD'}]

> Finished chain.
PaymentService is maintained by TeamD.PaymentService is maintained by TeamD.

> Finished chain.
PaymentService is maintained by TeamD.


In [46]:
response = mrkl.run("Which tasks have optimization in their description?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Tasks` with `Which tasks have optimization in their description?`




> Entering new RetrievalQA chain...

> Finished chain.
There are two tasks with optimization in their description:

1. Optimize PaymentService
2. Optimize AuthServiceThere are two tasks with optimization in their description:

1. Optimize PaymentService
2. Optimize AuthService

> Finished chain.
There are two tasks with optimization in their description:

1. Optimize PaymentService
2. Optimize AuthService
